In [7]:
import json
import time
import requests

In [19]:
import pprint as pp

f= open('data/planningarea_latlon.csv','r')
planningarearaw = f.read()
f.close()

processed1 = planningarearaw.split("\n")
processed1.pop(0)
planningarea = {}
names = []
for i in processed1:
    x = i.split(",")
    planningarea[x[0]] = {'name': x[1], 'lon': x[2], 'lat': x[3]}
    names.append(x[1])

pp.pprint(len(planningarea))

55


In [20]:
print(names)

['ANG MO KIO', 'BEDOK', 'BUKIT BATOK', 'BOON LAY', 'BUKIT MERAH', 'BUKIT PANJANG', 'BISHAN', 'BUKIT TIMAH', 'CHANGI BAY', 'CENTRAL WATER CATCHMENT', 'CHANGI', 'CHOA CHU KANG', 'CLEMENTI', 'DOWNTOWN CORE', 'GEYLANG', 'HOUGANG', 'JURONG EAST', 'JURONG WEST', 'KALLANG', 'LIM CHU KANG', 'MANDAI', 'MARINA EAST', 'MARINE PARADE', 'MARINA SOUTH', 'MUSEUM', 'NORTH-EASTERN ISLANDS', 'NEWTON', 'NOVENA', 'ORCHARD', 'OUTRAM', 'PUNGGOL', 'PAYA LEBAR', 'PIONEER', 'PASIR RIS', 'QUEENSTOWN', 'ROCHOR', 'RIVER VALLEY', 'SEMBAWANG', 'SENGKANG', 'SERANGOON', 'SOUTHERN ISLANDS', 'SUNGEI KADUT', 'SELETAR', 'SIMPANG', 'SINGAPORE RIVER', 'STRAITS VIEW', 'TENGAH', 'TAMPINES', 'TANGLIN', 'TOA PAYOH', 'TUAS', 'WESTERN WATER CATCHMENT', 'WOODLANDS', 'WESTERN ISLANDS', 'YISHUN']


In [4]:
pp.pprint(planningarea)

{'AM': {'lat': '1.38009', 'lon': '103.8421', 'name': 'ANG MO KIO'},
 'BD': {'lat': '1.32405', 'lon': '103.92841', 'name': 'BEDOK'},
 'BK': {'lat': '1.35565', 'lon': '103.75474', 'name': 'BUKIT BATOK'},
 'BL': {'lat': '1.3148', 'lon': '103.70223', 'name': 'BOON LAY'},
 'BM': {'lat': '1.27548', 'lon': '103.82311', 'name': 'BUKIT MERAH'},
 'BP': {'lat': '1.36674', 'lon': '103.77304', 'name': 'BUKIT PANJANG'},
 'BS': {'lat': '1.35496', 'lon': '103.84042', 'name': 'BISHAN'},
 'BT': {'lat': '1.32999', 'lon': '103.79069', 'name': 'BUKIT TIMAH'},
 'CB': {'lat': '1.32718', 'lon': '104.02612', 'name': 'CHANGI BAY'},
 'CC': {'lat': '1.37666',
        'lon': '103.80119',
        'name': 'CENTRAL WATER CATCHMENT'},
 'CH': {'lat': '1.35081', 'lon': '103.99782', 'name': 'CHANGI'},
 'CK': {'lat': '1.38555', 'lon': '103.74719', 'name': 'CHOA CHU KANG'},
 'CL': {'lat': '1.3176', 'lon': '103.76074', 'name': 'CLEMENTI'},
 'DT': {'lat': '1.28508', 'lon': '103.8543', 'name': 'DOWNTOWN CORE'},
 'GL': {'lat':

In [4]:
def writeOutp(res):
    # count breakdown by rail/road/unknown
#     data = {}
#     orig = res[0]['event']['origin_planningarea']
#     date = res[0]['timestamp'][:9]
#     time = None
#     for journey in res:
#         ev = journey['event']
#         dest = ev['destination_planningarea']
#         thisTime = journey['timestamp'][11:19]
#         if thisTime != time:
#             time = thisTime
#             data[time] = {}
#         if dest not in data[time]:
#             data[time][dest] = {'ROAD':0, 'TRAIN':0, 'UNKNOWN':0}
#         data[time][dest][ev['dominant_mode']] = ev['longSum_total_records']
    
#     pp.pprint(data)
    
#     outp = ''
#     for time in data:
#         for dest in data[time]:
#             s = []
#             s.append(time)
#             s.append(date)
#             s.append(orig)
#             s.append(dest)
#             s.append(str(data[time][dest]['ROAD']))
#             s.append(str(data[time][dest]['TRAIN']))
#             s.append(str(data[time][dest]['UNKNOWN']))
            
#             outp += ','.join(s)+'\n'
#     outf.write(outp)
    
    # total count only
    for journey in res:
        if journey['event']['destination_planningarea'] == 'NA':
            continue
        ev = journey['event']
        s = []
        s.append(journey['timestamp'][:9])
        s.append(journey['timestamp'][11:19])
        
        o = ev['origin_planningarea']
        s.append(o)
        s.append(planningarea[o]['name'])
        s.append(planningarea[o]['lon'])
        s.append(planningarea[o]['lat'])
        
        d = ev['destination_planningarea']
        s.append(d)
        s.append(planningarea[d]['name'])
        s.append(planningarea[d]['lon'])
        s.append(planningarea[d]['lat'])
        
        s.append(str(ev['longSum_total_records']))
        
        outstr = ','.join(s)+'\n'
        outf.write(outstr)
        if ev['longSum_total_records'] > 40:
            outf_filter.write(outstr)

    print('output written for: ' + res[0]['event']['origin_planningarea'])


In [5]:
queryBody = {
    "date": "2018-02-22",
    "timeSeriesReference": "origin",
    "location": {
        "locationType": "locationHierarchyLevel",
        "levelType": "origin_planningarea",
        "id": "AM"
    },
    "queryGranularity": {
        "type": "period",
        "period": "PT1H"
    },
    "dimensionFacets": ["destination_planningarea"],
    "aggregations": [
    {
        "metric": "unique_agents",
        "type": "hyperUnique"
    },
    {
        "metric": "total_records",
        "type": "longSum"
    }
    ]
}

outf = open("data/collectedOD_all.csv", "w")
outf_filter = open("data/collectedOD_filter.csv", "w")
outf.write("date,time,origin,origin_name,origin_lon,origin_lat,destination,dest_name,dest_lon,dest_lat,total_count\n")
outf_filter.write("date,time,origin,origin_name,origin_lon,origin_lat,destination,dest_name,dest_lon,dest_lat,total_count\n")

for area in planningarea:
    print("getting data for: " + area)
    code = area
    queryBody["location"]["id"] = code
    
    token = ""
    queryResponse = requests.post("https://apistore.datasparkanalytics.com:443/odmatrix/v3/query",
     data = json.dumps(queryBody),
     headers = {
       'Authorization': 'Bearer ' + token,
       'Content-Type': 'application/json'
     }
    )

    res = queryResponse.json()
    writeOutp(res)
    time.sleep(10)

outf.close()
outf_filter.close()

getting data for: AM
output written for: AM
getting data for: BD
output written for: BD
getting data for: BK
output written for: BK
getting data for: BL
output written for: BL
getting data for: BM
output written for: BM
getting data for: BP
output written for: BP
getting data for: BS
output written for: BS
getting data for: BT
output written for: BT
getting data for: CB
output written for: CB
getting data for: CC
output written for: CC
getting data for: CH
output written for: CH
getting data for: CK
output written for: CK
getting data for: CL
output written for: CL
getting data for: DT
output written for: DT
getting data for: GL
output written for: GL
getting data for: HG
output written for: HG
getting data for: JE
output written for: JE
getting data for: JW
output written for: JW
getting data for: KL
output written for: KL
getting data for: LK
output written for: LK
getting data for: MD
output written for: MD
getting data for: ME
output written for: ME
getting data for: MP
output writ

In [ ]:
for journey in res:
    ev = journey["event"]
    if ev["longSum_total_records"] == 40:
#     if True:
        print(journey["timestamp"][11:13] +"h "+ ev["destination_planningarea"]+" "+ev["dominant_mode"]+": "+str(ev["longSum_total_records"])+" "+str(ev["longSum_sum_duration"]/ev["longSum_total_records"]/ev["longSum_total_records"]))
        
        